In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Input Data Import and Staging
# Load both rows from the CSV
data = np.loadtxt("actual_power_bills.csv", delimiter=",")
# Split into two variables
monthly_bills = [0]
monthly_temp = [1]

# 1. Simulated 3 years of power bills
monthly_bills = [
    110, 115, 120, 140, 160, 180, 200, 195, 175, 150, 130, 115,
    112, 117, 125, 145, 165, 185, 205, 200, 180, 155, 135, 118,
    114, 119, 128, 148, 168, 188, 210, 205, 185, 160, 140, 120
]

# 2. Simulated average temperature (F) for the same months
# Hotter in Jul-Aug, colder in Jan-Feb, mild in Spring/Fall
monthly_avg_temps = [
    30, 32, 45, 55, 65, 75, 85, 87, 70, 60, 50, 35,   # Year 1
    28, 30, 43, 54, 66, 76, 88, 89, 72, 61, 49, 33,   # Year 2
    29, 31, 44, 56, 67, 77, 86, 88, 71, 59, 48, 34    # Year 3
]

# 3. Create date index starting Jan 2021
dates = pd.date_range(start='2021-01-01', periods=len(monthly_bills), freq='MS')
bills_series = pd.Series(monthly_bills, index=dates)
temps_series = pd.Series(monthly_avg_temps, index=dates)
exog_train = temps_series.to_frame()

# 4. Fit SARIMAX model with temperature as exogenous vairable
model = SARIMAX(bills_series,
                exog=exog_train,
                order=(1,1,1),
                seasonal_order=(1,1,1,12),
                enforce_stationarity=False,
                enforce_invertibility=False
                )
results = model.fit(disp=False)


In [ ]:

# 5. Forecast next 12 months (assume temperature continues seasonally)
future_temps = [32, 35, 46, 57, 68, 78, 89, 90, 73, 62, 51, 36] # Forecasted temps
future_index = pd.date_range(start=dates[-1] + pd.offsets.MonthBegin(), periods=12, freq='MS')
future_temps_series = pd.Series(future_temps, index=future_index).to_frame()
forecast = results.get_forecast(steps=12, exog=future_temps_series)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# 6. Plot the forecast
plt.figure(figsize=(12,6))
plt.plot(bills_series, label='Observed Bills', color='blue')
plt.plot(future_index, forecast_mean, label='Forecasted Bills', color='red')
plt.fill_between(future_index, forecast_ci.iloc[:,0], forecast_ci.iloc[:,1],
                 color='pink', alpha=0.3)

plt.title('Power Bill Forecast with Temperature as Exogenous Variable')
plt.xlabel('Date')
plt.ylabel('Bill Amount ($)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 7. Print forecasted bills
print("Forecasted power bills with temperature:")
for date, value in zip(future_index, forecast_mean):
    print(f"{date.strftime('%B %Y')}: ${value:.2f}")
